In [1]:
pip install sec_edgar_downloader

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from sec_edgar_downloader import Downloader
from bs4 import BeautifulSoup
import os
import re

def download_latest_filing(ticker: str = "NVDA", form_type: str = "10-Q") -> str:
    dl = Downloader("dvschenone@ucsd.edu", "sec_data")
    dl.get(form_type, ticker)
    return get_latest_filing_file(ticker, form_type)


def get_latest_filing_file(ticker: str, form_type: str) -> str:
    filings_path = os.path.join("sec_data", "sec-edgar-filings", ticker, form_type)
    latest_file = None
    latest_mtime = 0

    for root, _, files in os.walk(filings_path):
        for file in files:
            if file.endswith(".html") or file.endswith(".txt"):
                file_path = os.path.join(root, file)
                mtime = os.path.getmtime(file_path)
                if mtime > latest_mtime:
                    latest_file = file_path
                    latest_mtime = mtime
    return latest_file


def extract_item_1a(file_path: str) -> str:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        content = f.read()

    # If HTML-like, use BeautifulSoup
    if "<html" in content.lower():
        soup = BeautifulSoup(content, "html.parser")
        text = soup.get_text(separator="\n")
    else:
        text = content

    # Normalize newlines and search
    text = re.sub(r'\n+', '\n', text)
    pattern = r'(?i)(item\s+1A\.?\s*[-–:]?\s*Risk\s+Factors)(.*?)(item\s+1B\.?|item\s+2\.)'
    match = re.search(pattern, text, re.DOTALL)

    if match:
        return match.group(2).strip()
    else:
        return "Item 1A section not found."


# Example
html_path = download_latest_filing("NVDA", "10-Q")

if html_path is None:
    print("❌ Could not locate any downloaded filing.")
else:
    item_1a_text = extract_item_1a(html_path)
    print(item_1a_text[:1000])



❌ Could not locate any downloaded HTML filing.


TypeError: expected str, bytes or os.PathLike object, not NoneType